# Notebook for running testing

## Import settings from settings notebook

In [1]:
%run MP1_settings.ipynb

## Load all files from testing data folder

In [2]:
line = []
freq = []

file_list = glob.glob(r'data\test_data\*.csv')
for filename in file_list:
    df = pd.read_csv (filename, delimiter=r'\s+,', decimal=',', skiprows=1, header=None, names=colnames, engine='python')
    line.append(np.array(df[Y_NAME]))
    freq.append(np.array(df[X_NAME]))
    

n = len(freq)
n_freq = len(freq[0])
line = np.array(line)
freq = np.array(freq)

In [3]:
df = pd.read_csv (parameter_path, delimiter=r',', decimal='.')

In [4]:
basefreq = np.array(df['basefreq'])
baseline = np.array(df['baseline'])

n_freq_base = len(basefreq)

vert_shift_spectrum = np.zeros((ver_DIM, n_freq_base)) 

for k in range(ver_DIM):
    vert_shift_spectrum[k] = np.array(df[f'parameter_{k}'])

In [5]:
freq_shifted = np.zeros((n,n_freq))

for i in range(n):
    def min_func(p):
        line_int = interpolate.interp1d(horizontal_shift(freq[i], p), line[i], kind=INT_METHOD, bounds_error=False)(basefreq)
        return 1 - CC_btw(line_int, baseline)

    popt = minimize(min_func, hor_PARAM_INIT, method=MIN_METHOD, tol=1e-12, bounds=hor_PARAM_RANGE)

    freq_shifted[i] = horizontal_shift(freq[i], popt.x)

    clear_output(wait=True)
    print(f'Horizontal shift assessment')
    print(f'{i+1}/{n}')

Horizontal shift assessment
320/320


In [6]:
line_fit = np.zeros((n,n_freq_base))

for i in range(n):
    line_fit[i] = interpolate.interp1d(freq_shifted[i], line[i], kind=INT_METHOD, bounds_error=False)(basefreq)

In [7]:
line_guessed = np.zeros((n,n_freq_base))
T_guessed = np.zeros(n)

for i in range(n):
    def min_func(p):
        return RMSD_btw(baseline, vertical_shift(line_fit[i], vert_shift_spectrum, *p))


    popt = minimize(min_func, [30], method=MIN_METHOD, tol=1e-12, bounds=np.array([[20,40]]))
    T_guessed[i] = popt.x

    line_guessed[i] = vertical_shift(line_fit[i], vert_shift_spectrum,  T_guessed[i])

    clear_output(wait=True)
    print(f'Vertical shift assessment')
    print(f'{i+1}/{n}')

Vertical shift assessment
320/320


# Compute final result
The result is the RMSD between each horizontally and vertically shifted line. If the training was successful, there should be a threshhold between measurements with structural damage and no structural damage.

In [8]:
rmsd = []
for i in range(n):
    rmsd.append(RMSD_btw(line_guessed[i], baseline))

# Save result
The result can be read and analyzed later. Additionally the predicted Temperature is saved.

In [9]:
header = ['file', 'RMSD', 'T_predicted']

with open(result_path, 'w', encoding='UTF8') as f:
    writer = csv.writer(f, delimiter =',')
    f.truncate(0)
    # Write the header
    writer.writerow(header)
    
    for i in range(n):
        data = [file_list[i], rmsd[i], T_guessed[i]]
        # Write data
        writer.writerow(data)

print('Result saved to {}'.format(result_path))

Result saved to data\result.dat
